# Exploring and visualizing DICOM SEG
The purpose of this notebook is to provide guidance when attempting to explore and visualize certain DICOM data sets as provided by [AIDA](https://datahub.aida.scilifelab.se/). In particular, datasets containing DICOM SEG. For example, the following datasets:
* [CTPA](https://datahub.aida.scilifelab.se/)
* [CTPEL](https://datahub.aida.scilifelab.se/10.23698/aida/ctpel)
* [DRLI](https://datahub.aida.scilifelab.se/10.23698/aida/drli)
* [DRSKE](https://datahub.aida.scilifelab.se/10.23698/aida/drske)

In the following cells, it is assumed that one examination from any of these examinations are available locally.

## Preparations

In [1]:
# Imports
%matplotlib inline
from pathlib import Path
import pydicom
import numpy as np
from skimage import measure
import matplotlib.pyplot as plt 
from ipywidgets.widgets import *

from lib.helper import read_and_sort_dicom_files

Specify data to explore and visualize by providing a path to a folder. It is assumed that the specified folder contains a single examination and that all relevant files end with `.dcm`.

In [2]:
folder = Path("/mnt/c/Users/da-for/Downloads/0FC3188AAA7E6851")

Loop over all files to understand the content and to create a mapping structure.


In [3]:
seg_files = []
sop_uid_to_file = {}
sop_uid_to_series_uid = {}
series_uid_to_sop_uids = {}

for dcm_file in folder.rglob("*.dcm"):
    ds = pydicom.read_file(dcm_file)
    if ds.Modality == "SEG":
        seg_files.append(dcm_file)
    
    sop_uid_to_file[ds.SOPInstanceUID] = dcm_file
    sop_uid_to_series_uid[ds.SOPInstanceUID] = ds.SeriesInstanceUID
    if ds.SeriesInstanceUID not in series_uid_to_sop_uids:
        series_uid_to_sop_uids[ds.SeriesInstanceUID] = []
    series_uid_to_sop_uids[ds.SeriesInstanceUID].append(ds.SOPInstanceUID)

## DICOM SEG
A DICOM SEG object contains information about a set of segmented objects as derived from a set of referenced DICOM instances. Most commonly, the segmentations are binary, but they can also be fractional. A tricky aspect of the DICOM SEG objects is that they are multi-frame objects, i.e., instead of one DICOM SEG object with a single segmentation mask per image and segmented object in, for example, a CT stack, a single DICOM SEG object contains all the segmentation masks for all segmented objects.

In the following, we assume that at least one SEG file was found and that we use the first one.

In [4]:
seg_file = seg_files[0]
seg_ds = pydicom.read_file(seg_file)
print(f"The SEG file {seg_file} is selected for exploration")

The SEG file /mnt/c/Users/da-for/Downloads/0FC3188AAA7E6851/im_3/x0000.dcm is selected for exploration


In the following cells, the content of the SEG file will be explored step by step.

In [5]:
print(f"The SEG file contains {len(seg_ds.SegmentSequence)} segments")

The SEG file contains 5 segments


The first segment has the following description/content:

In [6]:
print(f"The first segmentation has the following description/content:")
print(f"Anatomic Region Sequence to describe the anatomical region of the segmentation")
print(seg_ds.SegmentSequence[0].AnatomicRegionSequence[0])
print(f"")
print(f"Segmented Property Category Code Sequence to describe the general category of the segmentation")
print(seg_ds.SegmentSequence[0].SegmentedPropertyCategoryCodeSequence[0])
print(f"")
print(f"Segmented Property Type Code Sequence to describe the specific propery of the segmentation, what the segmented voxels represent")
print(seg_ds.SegmentSequence[0].SegmentedPropertyTypeCodeSequence[0])
print(f"")
print(f"Some additional attributes include:")
for attribute in ["SegmentNumber", "SegmentLabel", "SegmentDescription", "SegmentAlgorithmType", "SegmentAlgorithmName"]:
    print(seg_ds.SegmentSequence[0].data_element(attribute))

The first segmentation has the following description/content:
Anatomic Region Sequence to describe the anatomical region of the segmentation
(0008, 0100) Code Value                          SH: 'R-FAB57'
(0008, 0102) Coding Scheme Designator            SH: 'SRT'
(0008, 0104) Code Meaning                        LO: 'Abdomen and Pelvis'

Segmented Property Category Code Sequence to describe the general category of the segmentation
(0008, 0100) Code Value                          SH: 'T-D000A'
(0008, 0102) Coding Scheme Designator            SH: 'SRT'
(0008, 0104) Code Meaning                        LO: 'Anatomical Structure'

Segmented Property Type Code Sequence to describe the specific propery of the segmentation, what the segmented voxels represent
(0008, 0100) Code Value                          SH: 'T-12710'
(0008, 0102) Coding Scheme Designator            SH: 'SRT'
(0008, 0104) Code Meaning                        LO: 'Femur'
(0062, 0011)  Segmented Property Type Modifier Code Seque

The SEG file also contains information about the frames.

The `NumberOfFrames` specifies the number of frames contained in the `PixelData` attribute.

In [7]:
print(seg_ds.data_element("NumberOfFrames"))

(0028, 0008) Number of Frames                    IS: '901'


Each frame is defined by:

In [8]:
print(seg_ds.data_element("Rows"))
print(seg_ds.data_element("Columns"))

(0028, 0010) Rows                                US: 512
(0028, 0011) Columns                             US: 512


The `SharedFunctionalGroupsSequence` specifies information shared for all frames:

In [9]:
print(f"{seg_ds.SharedFunctionalGroupsSequence[0]}")

(0020, 9116)  Plane Orientation Sequence  1 item(s) ---- 
   (0020, 0037) Image Orientation (Patient)         DS: [1.000000e+000, 0.000000e+000, 0.000000e+000, 0.000000e+000, 1.000000e+000, 0.000000e+000]
   ---------
(0028, 9110)  Pixel Measures Sequence  1 item(s) ---- 
   (0018, 0050) Slice Thickness                     DS: '0.7999878'
   (0018, 0088) Spacing Between Slices              DS: '0.7999878'
   (0028, 0030) Pixel Spacing                       DS: [7.812500e-001, 7.812500e-001]
   ---------


The `PerFrameFunctionalGroupsSequence` specifies information per frame:

In [10]:
print(f"{seg_ds.PerFrameFunctionalGroupsSequence[0]}")

(0008, 9124)  Derivation Image Sequence  1 item(s) ---- 
   (0008, 2112)  Source Image Sequence  1 item(s) ---- 
      (0008, 1150) Referenced SOP Class UID            UI: CT Image Storage
      (0008, 1155) Referenced SOP Instance UID         UI: 1.3.6.1.4.1.9328.50.4.13397
      (0040, a170)  Purpose of Reference Code Sequence  1 item(s) ---- 
         (0008, 0100) Code Value                          SH: '121322'
         (0008, 0102) Coding Scheme Designator            SH: 'DCM'
         (0008, 0104) Code Meaning                        LO: 'Source image for image processing operation'
         ---------
      ---------
   (0008, 9215)  Derivation Code Sequence  1 item(s) ---- 
      (0008, 0100) Code Value                          SH: '113076'
      (0008, 0102) Coding Scheme Designator            SH: 'DCM'
      (0008, 0104) Code Meaning                        LO: 'Segmentation'
      ---------
   ---------
(0020, 9111)  Frame Content Sequence  1 item(s) ---- 
   (0020, 9157) Dimen

All this can now be used to visualize the contained segments, but first we will map all the segments/frames to the respective `ReferencedSOPInstanceUID`.

In [11]:
sop_uid_to_frame_indices = dict()
SERIES_INSTANCE_UID = None
for ind, frame in enumerate(seg_ds.PerFrameFunctionalGroupsSequence):
    if not SERIES_INSTANCE_UID:
        SERIES_INSTANCE_UID = sop_uid_to_series_uid[frame.DerivationImageSequence[0].SourceImageSequence[0].ReferencedSOPInstanceUID]
    if frame.DerivationImageSequence[0].SourceImageSequence[0].ReferencedSOPInstanceUID not in sop_uid_to_frame_indices:
        sop_uid_to_frame_indices[frame.DerivationImageSequence[0].SourceImageSequence[0].ReferencedSOPInstanceUID] = []
    sop_uid_to_frame_indices[frame.DerivationImageSequence[0].SourceImageSequence[0].ReferencedSOPInstanceUID].append(ind)

sop_uids = series_uid_to_sop_uids[SERIES_INSTANCE_UID]
dcm_files = [sop_uid_to_file[sop_uid] for sop_uid in sop_uids]
sorted_ds = read_and_sort_dicom_files(dcm_files)

Visualize the segments together with the referenced images.

In [12]:
# Define a function to use to show images and associated segments
def show_image_and_segments(ind, sorted_ds, sop_uid_to_frame_indices, frames):
    image_ds = sorted_ds[ind]
    sop_uid = image_ds.SOPInstanceUID
    fig, ax = plt.subplots(1, figsize = (9,9), dpi=100)
    ax.imshow(image_ds.pixel_array, cmap=plt.cm.gray)
    frame_indices = []
    if sop_uid in sop_uid_to_frame_indices:
        frame_indices = sop_uid_to_frame_indices[sop_uid]
    for frame_index in frame_indices:
        segmentation_mask = frames[frame_index,:,:]
        contours = measure.find_contours(segmentation_mask)
        for contour in contours:
            ax.plot(contour[:, 1], contour[:, 0], linewidth=1, color=[1,0,0])    
    plt.show()
    return ind

In [13]:
interact(
    show_image_and_segments, 
    ind=(0, len(sorted_ds)-1), 
    sorted_ds=fixed(sorted_ds), 
    sop_uid_to_frame_indices=fixed(sop_uid_to_frame_indices),
    frames=fixed(seg_ds.pixel_array)
)

interactive(children=(IntSlider(value=267, description='ind', max=534), Output()), _dom_classes=('widget-inter…

<function __main__.show_image_and_segments(ind, sorted_ds, sop_uid_to_frame_indices, frames)>